In [47]:
import pandas as pd
import numpy as np

In [48]:
folder = 'C:\\Users\\user\\Documents\\LightHouseLabs\\W09'

In [49]:
submission_file = pd.read_csv(folder + "\\submission.csv")

In [50]:
submission_file['site']=submission_file['site_path_timestamp'].str.split("_").str[0]

In [51]:
submission_file.head()

,site_path_timestamp,floor,x,y,site
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,90.939186,94.909570,5a0546857ecc773753327266
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,91.923904,94.103546,5a0546857ecc773753327266
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,93.637100,103.851870,5a0546857ecc773753327266
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,91.419460,103.945190,5a0546857ecc773753327266
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,95.712600,107.202095,5a0546857ecc773753327266


In [52]:
submission_file.shape

(10133, 5)

In [53]:
base_path = 'E:\\indoor-location-navigation\\'

In [54]:
#Generating waypoints dataframe
#Iterating all training files

In [55]:
import os

train_files = 'C:\\Users\\user\\Documents\\LightHouseLabs\\W09\\Rev 1 train files'

waypoints_total = pd.DataFrame(columns=['x', 'y', 'f', 'site'])

for filename in os.listdir(train_files):
    if filename.endswith(".csv") and 'train' in filename:
        waypoints_df= pd.read_csv(os.path.join(train_files, filename))
        waypoints_df=waypoints_df[['x','y', 'f']]
        waypoints_df['site']=filename.split('_')[0]
        waypoints_df.drop_duplicates(subset=['x', 'y', 'f'], inplace=True)
        waypoints_total = pd.concat([waypoints_total, waypoints_df], ignore_index=True)
    else:
        continue
waypoints_total.head(20)

,x,y,f,site
0,230.03738,153.49635,-1,5a0546857ecc773753327266
1,231.40290,158.41515,-1,5a0546857ecc773753327266
2,232.46200,164.41673,-1,5a0546857ecc773753327266
3,233.94418,171.41417,-1,5a0546857ecc773753327266
4,198.36833,163.52063,-1,5a0546857ecc773753327266
5,204.52890,162.11609,-1,5a0546857ecc773753327266
6,197.05070,158.34859,-1,5a0546857ecc773753327266
7,203.69691,157.82071,-1,5a0546857ecc773753327266
8,199.74606,168.01990,-1,5a0546857ecc773753327266
9,206.10878,167.22223,-1,5a0546857ecc773753327266


In [56]:
waypoints_total.shape

(25343, 4)

In [57]:
from scipy.spatial.distance import cdist

In [58]:
def add_xy(df):
    df['xy'] = [(x, y) for x,y in zip(df['x'], df['y'])]
    return df

In [59]:
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

In [60]:
def snap_to_grid(sub, threshold):
    """
    Snap to grid if within a threshold.
    
    x, y are the predicted points.
    x_, y_ are the closest grid points.
    _x_, _y_ are the new predictions after post processing.
    """
    sub['_x_'] = sub['x']
    sub['_y_'] = sub['y']
    sub.loc[sub['dist'] < threshold, '_x_'] = sub.loc[sub['dist'] < threshold]['x_']
    sub.loc[sub['dist'] < threshold, '_y_'] = sub.loc[sub['dist'] < threshold]['y_']
    return sub.copy()

In [61]:
submission_file = add_xy(submission_file)
waypoints_total = add_xy(waypoints_total)

In [62]:
ds=[]
for (site, myfloor), d in submission_file.groupby(['site','floor']):
    true_floor_locs = waypoints_total.loc[(waypoints_total['f'] == myfloor) &
                                          (waypoints_total['site'] == site)] \
        .reset_index(drop=True)
    if len(true_floor_locs) == 0:
        print(f'Skipping {site} {myfloor}')
        continue
    d['matched_point'] = [closest_point(x, list(true_floor_locs['xy'])) for x in d['xy']]
    d['x_'] = d['matched_point'].apply(lambda x: x[0])
    d['y_'] = d['matched_point'].apply(lambda x: x[1])
    ds.append(d)

sub = pd.concat(ds)

In [63]:
sub.head()

,site_path_timestamp,floor,x,y,site,xy,matched_point,x_,y_
17,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,156.88347,138.45499,5a0546857ecc773753327266,"(156.88347, 138.45498999999998)","(157.16997, 136.42146)",157.16997,136.42146
18,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,160.76607,139.68266,5a0546857ecc773753327266,"(160.76606999999998, 139.68266)","(161.38523999999998, 135.60426)",161.38524,135.60426
19,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,159.56511,140.03844,5a0546857ecc773753327266,"(159.56511, 140.03843999999998)","(157.16997, 136.42146)",157.16997,136.42146
20,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,157.75691,150.85129,5a0546857ecc773753327266,"(157.75691, 150.85128999999998)","(158.62456, 150.17136000000002)",158.62456,150.17136
21,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,158.81096,163.15610,5a0546857ecc773753327266,"(158.81096000000002, 163.1561)","(160.7603, 164.57616000000002)",160.76030,164.57616


In [64]:
sub.shape

(10133, 9)

In [65]:
# Calculate the distances
sub['dist'] = np.sqrt( (sub.x-sub.x_)**2 + (sub.y-sub.y_)**2 )

sub_pp = snap_to_grid(sub, threshold=12)

In [66]:
sub_pp.head()

,site_path_timestamp,floor,x,y,site,xy,matched_point,x_,y_,dist,_x_,_y_
17,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,156.88347,138.45499,5a0546857ecc773753327266,"(156.88347, 138.45498999999998)","(157.16997, 136.42146)",157.16997,136.42146,2.053613,157.16997,136.42146
18,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,160.76607,139.68266,5a0546857ecc773753327266,"(160.76606999999998, 139.68266)","(161.38523999999998, 135.60426)",161.38524,135.60426,4.125132,161.38524,135.60426
19,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,159.56511,140.03844,5a0546857ecc773753327266,"(159.56511, 140.03843999999998)","(157.16997, 136.42146)",157.16997,136.42146,4.338115,157.16997,136.42146
20,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,157.75691,150.85129,5a0546857ecc773753327266,"(157.75691, 150.85128999999998)","(158.62456, 150.17136000000002)",158.62456,150.17136,1.102325,158.62456,150.17136
21,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,158.81096,163.15610,5a0546857ecc773753327266,"(158.81096000000002, 163.1561)","(160.7603, 164.57616000000002)",160.76030,164.57616,2.411741,160.76030,164.57616


In [67]:
sub = sub_pp[['site_path_timestamp','floor','_x_','_y_']].rename(columns={'_x_':'x', '_y_':'y'})

In [68]:
sub.head()

,site_path_timestamp,floor,x,y
17,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,157.16997,136.42146
18,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,161.38524,135.60426
19,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,157.16997,136.42146
20,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,158.62456,150.17136
21,5a0546857ecc773753327266_0c06cc9f21d172618d74c...,-1,160.76030,164.57616


In [69]:
sub.to_csv('submission_snap_to_grid.csv', index=False)